<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Инструкция по выполнению проекта

- Загрузите и изучите данные.
- Ответьте на вопрос и обоснуйте решение.
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 - a. Изменится. Приведите примеры матриц.
 - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

- Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


In [2]:
# Открываем файл с данными
insurance = pd.read_csv('/datasets/insurance.csv')

In [3]:
# Первые 5 строк
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# Информация
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


5 столбцов и 5000 строк без пропущенных и нулевых значений, тип данных в признаках 'зарплата' и 'возраст' - вещественный, приведем к типу int.

In [5]:
insurance['Возраст'] = insurance['Возраст'].astype('int64')
insurance['Зарплата'] = insurance['Зарплата'].astype('int64')

In [6]:
# Размеры
insurance.shape

(5000, 5)

In [7]:
# Проверим на дубликаты
insurance.duplicated().sum()

153

Имеются 153 дубликата в данных, очистим данные от них.

In [8]:
insurance.drop_duplicates().reset_index(drop=True)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4842,0,28,35700,2,0
4843,0,34,52400,1,0
4844,0,20,33900,2,0
4845,1,22,32700,3,0


Можно видеть, что строк стало (5 000 - 153) 4 847

Имеем 5 признаков и 5 000 наблюдений. Пропущенных и нулевых значений в данных нет, тип данных в столбцах 'зарплата' и 'возраст' приведен к целочисленному типу данных. В данных были обнаружены и удалены дубликаты.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** 

Новая матрица признаков:  X' = X * P 

тогда формула обучения будет выглядеть так:  
$$
w' = ((XP)^T (XP))^{-1} (XP)^T y
$$

Мы знаем свойство транспонированной матрицы:
$$
(АВ)^T  = В^TА^T 
$$

Исходя из этого свойства, мы получим: 
$$
(XP)^T = Р^TХ^T
$$
и, следовательно, мы получим формулу такого вида:
$$
w' = (Р^TХ^TХР)^{-1}Р^TХ^Tу
$$

А так же, из условия мы знаем, что матрица Р - это обратимая матрица, а обратимая матрица -это квадратная матрица, для которой можно найти обратную, можно к формуле применить свойство обратимых матриц такого вида :
$$
(АВ)^{-1} = В^{-1}А^{-1}
$$

И вот эта часть в нашей формуле будет выгляет так: 

$$
(Р^TХ^TХР)^{-1} = Р^{-1}(Х^TХ)^{-1}(Р^T)^{-1}
$$

И получим:
$$
w' = Р^{-1}(Х^TХ)^{-1}(Р^T)^{-1}Р^TХ^Tу
$$

А так же нам известно свойство: произведение матрицы А на обратную ей матрицу А^{-1} дает единичную матрицу, следовательно 
$$
(Р^T)^{-1}Р^T = 1
$$

И получаем такое уравнение:
$$
w' = Р^{-1}(Х^TХ)^{-1}Х^Tу
$$

А вот эта часть 
$$
(Х^TХ)^{-1}Х^Tу = w
$$

Следовательно мы получаем такое различе в весах
$$
w' = Р^{-1}w
$$

Далее, у нас prediction а = Xw, подставим полученные значения. По условию Х = ХР, а w' мы получили ранее, следовательно 
$$
a' = ХРР^{-1}w
$$

Опять же 
$$
РР^{-1} = 1
$$
та как Р, по условию, обратимая

Подставим в формулу и получаем
$$
a' = Хw = а
$$

И это означает, что качество линейной регрессии не изменится. 

## Алгоритм преобразования

**Алгоритм**

Для защиты данных умножим матрицу признаков на обратимую матрицу. Раз она квадратная, и чтобы умножение состоялось необходимо условие, чтобы матрица была 4х4, так как признаков в матрице Х у нас 4(Признаки: пол, возраст, зарплата застрахованного, количество членов его семьи.).

**Обоснование**

Ранее в пункте 2 я обосновала, что качество линейной регрессии не изменится, если перемножить матрицу признаков на обратимую матрицу.

## Проверка алгоритма

In [9]:
# Обозначим признаки и целевой признак
features = insurance.drop(['Страховые выплаты'], axis=1)
target = insurance['Страховые выплаты']

Получим метрику качества линейной регрессии до преобразования.

In [10]:
# Создание класса LinearRegression()
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [11]:
# Обучение модели и проверка до изменений  
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))   

0.42494550308169177


In [12]:
# Создание обратимой квадратной матрицы 4х4 и проверка ее на обратимость
P = np.random.normal(1, 10000, size = (4,4))
P

array([[  -683.71041517,  -2186.67007093, -16468.61513985,
        -13011.17420133],
       [  1691.47232008,  -1608.61556894,   8120.66833215,
         -4772.16690649],
       [-13757.02537863,  -6143.17015622,   2654.85340919,
         -8107.79676157],
       [   490.13977002,  18415.41982275,   -958.71092645,
         15277.96033034]])

In [13]:
# Проверка ее на обратимость
P = np.linalg.inv(P)

In [14]:
# Умножим матрицу признаков на обратимую матрицу Р
XP =  pd.DataFrame(data = features.values.dot(P), columns = features.columns)

In [15]:
# Обучение модели и проверка после умножения матрицы признаков на матрицу Р
model = LinearRegression()
model.fit(XP, target)
predictions = model.predict(XP)
print(r2_score(target, predictions)) 

0.4249455030816911


Что и требовалось доказать. Кчество линейной регрессии не изменилось.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [х]  Весь код выполняется без ошибок
- [х]  Ячейки с кодом расположены в порядке исполнения
- [х]  Выполнен шаг 1: данные загружены
- [х]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [х]  Указан правильный вариант ответа
    - [х]  Вариант обоснован
- [х]  Выполнен шаг 3: предложен алгоритм преобразования
    - [х]  Алгоритм описан
    - [х]  Алгоритм обоснован
- [х]  Выполнен шаг 4: алгоритм проверен
    - [х]  Алгоритм реализован
    - [х]  Проведено сравнение качества моделей до и после преобразования